In [4]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("pinxau1000/radioml2018")

print("Path to dataset files:", path)

c:\Users\USER\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


100%|██████████| 18.0G/18.0G [15:05<00:00, 21.4MB/s]  

Extracting files...


Path to dataset files: C:\Users\USER\.cache\kagglehub\datasets\pinxau1000\radioml2018\versions\2


In [1]:
!dir C:\Users\USER\.cache\kagglehub\datasets\pinxau1000\radioml2018\versions\2

 Volume in drive C is Windows SSD
 Volume Serial Number is 00BC-1571

 Directory of C:\Users\USER\.cache\kagglehub\datasets\pinxau1000\radioml2018\versions\2

06/03/2025  04:24 PM    <DIR>          .
06/03/2025  04:22 PM    <DIR>          ..
06/03/2025  04:24 PM               246 classes-fixed.json
06/03/2025  04:24 PM             1,632 classes-fixed.txt
06/03/2025  04:24 PM               262 classes.txt
06/03/2025  04:24 PM               172 datasets.desktop
06/03/2025  04:24 PM    21,449,148,312 GOLD_XYZ_OSC.0001_1024.hdf5
06/03/2025  04:24 PM            20,993 LICENSE.TXT
               6 File(s) 21,449,171,617 bytes
               2 Dir(s)  889,175,351,296 bytes free


In [5]:
dataset_path = "C:\\Users\\USER\\.cache\\kagglehub\\datasets\\pinxau1000\\radioml2018\\versions\\2\\GOLD_XYZ_OSC.0001_1024.hdf5"

In [ ]:
!pip install torch seaborn

1. Load dataset dan preprocessing

In [7]:
!pip install torchsummary

In [9]:
from torchsummary import summary
import pickle
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, accuracy_score
import seaborn as sns

In [6]:
import h5py

# Misalkan kamu sudah memuat file HDF5
with h5py.File(dataset_path, 'r') as f:
    X = f['X'][:]  # Sinyal IQ
    Y = f['Y'][:]  # Label modulasi
    Z = f['Z'][:]  # Label SNR
    
    print(f"Shape of X (IQ data): {X.shape}")
    print(f"Shape of Y (Labels): {Y.shape}")
    print(f"Shape of Z (SNR): {Z.shape}")


Shape of X (IQ data): (2555904, 1024, 2)
Shape of Y (Labels): (2555904, 24)
Shape of Z (SNR): (2555904, 1)


2. Pendefinisian Class dan Objek

In [ ]:
class BasicBlock(nn.Module):
     """ Basic Residual Block (used in ResNet) """
     def __init__(self, in_channels, out_channels):
         super(BasicBlock, self).__init__()
         self.conv1 = nn.Conv1d(in_channels, out_channels, kernel_size=3, padding=1)
         self.bn1 = nn.BatchNorm1d(out_channels)
         self.relu = nn.ReLU()
         self.conv2 = nn.Conv1d(out_channels, out_channels, kernel_size=3, padding=1)
         self.bn2 = nn.BatchNorm1d(out_channels)
         self.shortcut = nn.Sequential()
         if in_channels != out_channels:
             self.shortcut = nn.Sequential(
                 nn.Conv1d(in_channels, out_channels, kernel_size=1),
                 nn.BatchNorm1d(out_channels)
             )

     def forward(self, x):
         out = self.relu(self.bn1(self.conv1(x)))
         out = self.bn2(self.conv2(out))
         out += self.shortcut(x)  # residual connection
         out = self.relu(out)
         return out

class CNNResNetModel(nn.Module):
    def __init__(self, num_classes=8):
        super(CNNResNetModel, self).__init__()
        self.conv1 = nn.Conv1d(2, 64, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm1d(64)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool1d(2)
        
        # ResNet blocks
        self.resnet_block1 = BasicBlock(64, 128)
        self.resnet_block2 = BasicBlock(128, 256)
        self.resnet_block3 = BasicBlock(256, 512)
        
        self.fc1 = nn.Linear(512 * 16, 1024)  # Adjust according to input length after pooling
        self.fc2 = nn.Linear(1024, num_classes)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        # CNN part
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.pool(x)  # shape becomes (batch_size, 64, 64)

        # ResNet part
        x = self.resnet_block1(x)  # (batch_size, 128, 64)
        x = self.pool(x)  # shape becomes (batch_size, 128, 32)
        
        x = self.resnet_block2(x)  # (batch_size, 256, 32)
        x = self.pool(x)  # shape becomes (batch_size, 256, 16)
        
        x = self.resnet_block3(x)  # (batch_size, 512, 16)

        # Flattening the output for fully connected layers
        x = torch.flatten(x, 1)

        # Fully connected layers
        x = self.fc1(x)
        x = self.dropout(x)
        x = self.fc2(x)
        return x

class RadioMLDataset(Dataset):
    """ Dataset for loading and preprocessing HDF5 file """
    def __init__(self, hdf5_path, classes):
        with h5py.File(hdf5_path, 'r') as f:
            self.X = f['X'][:]
            self.Y = f['Y'][:]
            self.SNR = f['Z'][:]
        self.classes = classes
        self.class_to_idx = {cls: idx for idx, cls in enumerate(classes)}
        
    def __len__(self):
        return len(self.Y)

    def __getitem__(self, idx):
        # Normalize IQ samples to range [-1, 1]
        iq_data = self.X[idx]  # normalize
        label = self.Y[idx]
        if isinstance(label, np.ndarray):
            label = np.argmax(label)
        iq_data_normalized = iq_data / np.max(np.abs(iq_data))
        iq_tensor = torch.FloatTensor(iq_data_normalized)
        label_tensor = torch.tensor(label, dtype=torch.long)
        return iq_tensor, label_tensor # torch.FloatTensor(iq_data), torch.tensor(label)

hdf5_path = dataset_path
classes = ['BPSK', 'QPSK', '8PSK', 'QAM16', 'QAM64', 'CPFSK', 'GFSK', '4PAM']

dataset = RadioMLDataset(hdf5_path, classes)

# Melihat contoh data
print(f"Total samples: {len(dataset)}")
print(f"Shape of first sample (IQ): {dataset[0][0].shape}")
print(f"Label for first sample: {dataset[0][1]}")

Total samples: 2555904
Label raw (before processing): [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Type of label: <class 'numpy.ndarray'>
Shape of first sample (IQ): torch.Size([1024, 2])
Label raw (before processing): [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Type of label: <class 'numpy.ndarray'>
Label for first sample: 0


3. Subprogram Training dan Evaluasi

In [ ]:
def train_model(model, train_loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

    return running_loss / total, correct / total

def evaluate_model(model, test_loader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    all_labels = []
    all_preds = []

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            running_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(predicted.cpu().numpy())

    return running_loss / total, correct / total, all_labels, all_preds

4. Program Utama untuk Training dan Evaluasi

In [ ]:
def main():
    # Load HDF5 data
    hdf5_path = 'dataset.hdf5'  # Path to your HDF5 file
    classes = ['BPSK', 'QPSK', '8PSK', 'QAM16', 'QAM64', 'CPFSK', 'GFSK', '4PAM']
    dataset = RadioMLDataset(hdf5_path, classes)

    # Split into train and test sets (80-20)
    train_size = int(0.8 * len(dataset))
    test_size = len(dataset) - train_size
    train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

    train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=256, shuffle=False)

    # Model, loss function, optimizer
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = CNNResNetModel(num_classes=len(classes)).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=1e-3)

    # Train the model
    num_epochs = 25
    for epoch in range(num_epochs):
        train_loss, train_acc = train_model(model, train_loader, criterion, optimizer, device)
        test_loss, test_acc, _, _ = evaluate_model(model, test_loader, criterion, device)

        print(f"Epoch [{epoch+1}/{num_epochs}], "
              f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, "
              f"Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.4f}")

    # Final evaluation on the test set
    _, _, true_labels, predicted_labels = evaluate_model(model, test_loader, criterion, device)
    cm = confusion_matrix(true_labels, predicted_labels)
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', xticklabels=classes, yticklabels=classes, cmap='Blues')
    plt.title('Confusion Matrix')
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.show()

if __name__ == '__main__':
    main()